In [60]:
import pandas as pd
import numpy as np

basepath = "E:/vsodataforSerena/"

t_address_cols = ["address_type","customer_no","street1",
                  "street2","city","state","postal_code","country"]
t_address = pd.read_csv(basepath + "t_address.csv", usecols = t_address_cols)
t_subcrip_cols = ["customer_no","tot_no_of_subscription_seats",
                  "date_of_last_seat_change","box1_tot_subscription_price", "priority_code"]
t_cust_subscription_summary = pd.read_csv(basepath + "t_cust_subscription_summary.csv", usecols = t_subcrip_cols)
t_customer_cols = ["customer_no","last_activity_dt",
                   "last_ticket_dt","inactive","inactive_reason"]
t_customer = pd.read_csv(basepath + "t_customer.csv", usecols = t_customer_cols)
t_order_cols = ["customer_no", "solicitor",
                "tot_ticket_purch_amt", "tot_ticket_return_amt"]
t_order = pd.read_csv(basepath + "t_order.csv", usecols = t_order_cols)

C:\Users\Serena\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Serena\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (0,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Let's start by looking at the tables one by one. Note that the target column will be t_customer["inactive"].

In [164]:
# 1.0 and 1 are active 2.0 and 2 are inactive
# note that 1.0 means it was entered as an int, 1 means entered as a string
#t_customer["Churn?"] = np.where(t_customer["inactive"] == 1.0 )
conditions = [(t_customer["inactive"] == "1"),(t_customer["inactive"] == 1),
              (t_customer["inactive"] == "2"),(t_customer["inactive"] == 2)]
churn = ["No", "No", "Yes", "Yes"]
t_customer["Churn?"] = np.select(conditions, churn, default = np.nan)
t_customer["Churn?"].value_counts()

No     559641
Yes     70018
nan     19726
Name: Churn?, dtype: int64

from t_address:
- number of times moved: t_address.groupby("customer_no").count()["address_type"] **
- postal code of most recent entry: for each customer: t_address[t_address["customer_no"] == 9]["postal_code"].iloc[-1] (ex 9)	add_num = t_address.customer_no.unique()
for i in add_num:
    t_address[t_address["customer_no"] == i]["postal_code"].iloc[-1]
- country of most recent entry: similar to postal code

from t_cust_subscription_summary:
- priority code (most recent): t_cust_subscription_summary[t_cust_subscription_summary["customer_no"] == x]["priority_code].iloc[-1] *
- most recent subscription seats
- most recent box 1 total
- total subscription seats
- total box 1 total
- is currently subscribed
- box 1 total in past 6 months *** do this later
- subscription seats in past 6 months *** do this later

from t_order:
- number of orders: like ** (total)
- most recent solicitor: like *
- total ticket purchase amount: sum of all orders
- total return amount: sum 
- most recent purchase amount
- most recent return amount
- total purchase amount in past 6 months *** do this later
- number of orders in past 6 months *** do this later
- return amount in past 6 months *** do this later

from t_cust:
- inactive_reason

In [3]:
def recents(unique_customers, col, df):
    col_output = []
    for i in unique_customers: 
        try:
            col_output.append(df[df["customer_no"] == i][col].iloc[-1])
        except IndexError:
            continue
    col_output_temp = list(zip(list(unique_customers),col_output)) # list of tuples customer_no, most recent postal_code
    # make into series
    col_output_temp2 = list(zip(*col_output_temp))
    cust_w_col_output = pd.Series(col_output_temp2[1], index = col_output_temp2[0])
    return cust_w_col_output

In [4]:
def totals(unique_customers, col, df):
    total = []
    for i in unique_customers: 
        try:
            total.append(sum((df[df["customer_no"] == i][col])))
        except IndexError:
            continue
    col_output_temp = list(zip(list(unique_customers),total))
    col_output_temp2 = list(zip(*col_output_temp))
    cust_col_sum = pd.Series(col_output_temp2[1], index = col_output_temp2[0])
    return cust_col_sum

Current features:

From t_address: Number of times moved, most recent postal code, most recent country

From t_cust_subscription_summary: Most recent priority code, most recent subscription seats, most recent total purchase, lifetime seats total, lifetime box total
- don't need "is_subbed" since it's just based off of other columns

From t_order: total orders, most recent solicitor, lifetime purchase total, lifetime return total, most recent return, most recent purchase

From t_cust: inactive_reason

15 features in total.

Later it'd be useful to look at activity within the past 6 months, but due to how these dataframes and done, it's a bit difficult to do this. So do preliminiary ML stuff first.

In [179]:
t_cust = t_customer.copy()
t_cust.set_index("customer_no", inplace = True)
t_cust.head()

,last_activity_dt,last_ticket_dt,inactive,inactive_reason,Churn?
customer_no,,,,,
0,NaN,NaN,1,NaN,No
9,2016-08-24 16:48:48.000,2016-05-08 22:54:56.000,1,NaN,No
500,2013-08-29 16:03:04.000,2011-07-20 15:38:43.103,1,NaN,No
501,2013-08-29 17:14:43.100,2011-07-20 14:55:01.383,1,NaN,No
502,2016-08-03 17:07:26.000,2011-07-04 16:08:49.000,1,NaN,No


In [181]:
no_idx = list(t_cust[t_cust["Churn?"] == "No"].index)
yes_idx = list(t_cust[t_cust["Churn?"] == "Yes"].index)

9

In [182]:
# Process t_address
t_address_nochurn = t_address[t_address["customer_no"].isin(no_idx)]
t_address_yeschurn = t_address[t_address["customer_no"].isin(yes_idx)]
t_address_temp = t_address_nochurn.append(t_address_yeschurn)
print("Churn: " + str(len(t_address_yeschurn)) +"\nNo Churn: " + str(len(t_address_nochurn)))

Churn: 38330
No Churn: 277232


In [183]:
t_sub_nochurn = t_cust_subscription_summary[t_cust_subscription_summary["customer_no"].isin(no_idx)]
t_sub_yeschurn = t_cust_subscription_summary[t_cust_subscription_summary["customer_no"].isin(yes_idx)]
t_sub_temp = t_sub_nochurn.append(t_sub_yeschurn)
print("Churn: " + str(len(t_sub_yeschurn)) +"\nNo Churn: " + str(len(t_sub_nochurn)))

Churn: 1791
No Churn: 75641


In [184]:
t_order_nochurn = t_order[t_order["customer_no"].isin(no_idx)]
t_order_yeschurn = t_order[t_order["customer_no"].isin(yes_idx)]
t_order_temp = t_order_nochurn.append(t_order_yeschurn)
print("Churn: " + str(len(t_order_yeschurn)) +"\nNo Churn: " + str(len(t_order_nochurn)))

Churn: 6662
No Churn: 319084


In [185]:
# Get union of indices
address_cnum = t_address_temp.set_index("customer_no")
order_cnum = t_order_temp.set_index("customer_no")
sub_cnum = t_sub_temp.set_index("customer_no")
a_cnum = list(address_cnum.index)
o_cnum = list(order_cnum.index)
s_cnum = list(sub_cnum.index)
indices = set.intersection(set(a_cnum),set(o_cnum),set(s_cnum))

In [188]:
address_temp = t_address[t_address["customer_no"].isin(indices)] 
order_temp = t_order[t_order["customer_no"].isin(indices)] 
sub_temp = t_cust_subscription_summary[t_cust_subscription_summary["customer_no"].isin(indices)]
# different lengths but all have 7505 unique customer numbers, yay

In [189]:
# Now process as before
# t_address_temp
unique_customers = address_temp.customer_no.unique()
num_moves = address_temp.groupby("customer_no").count()["address_type"]
cust_postalcodes = recents(unique_customers, "postal_code", address_temp)
cust_countries = recents(unique_customers, "country", address_temp)

In [190]:
# t_cust_subscription_summary 
unique_custs_sub = sub_temp.customer_no.unique()
cust_prioritycode = recents(unique_custs_sub, "priority_code", sub_temp)
cust_prioritycode.fillna(0, inplace = True)
# dfs
recent_seats = recents(unique_custs_sub, "tot_no_of_subscription_seats", sub_temp)
recent_sub_price = recents(unique_custs_sub, "box1_tot_subscription_price", sub_temp)
total_seats = totals(unique_custs_sub, "tot_no_of_subscription_seats", sub_temp)
total_sub_price = totals(unique_custs_sub, "box1_tot_subscription_price", sub_temp)

In [191]:
# t_order
order_temp["tot_ticket_return_amt"] = abs(order_temp["tot_ticket_return_amt"])
unique_cust_order = order_temp.customer_no.unique()
# dfs
num_unique_orders = order_temp.groupby("customer_no").count()["solicitor"]
recent_solicitor = recents(unique_cust_order, "solicitor", order_temp)
recent_purchase_amt = recents(unique_cust_order, "tot_ticket_purch_amt", order_temp)
recent_return_amt = recents(unique_cust_order, "tot_ticket_return_amt", order_temp)
total_purchase_amt = totals(unique_cust_order, "tot_ticket_purch_amt", order_temp)
total_return_amt = totals(unique_cust_order, "tot_ticket_return_amt", order_temp)

C:\Users\Serena\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [192]:
from_address = pd.concat([num_moves, cust_postalcodes, cust_countries], axis = 1)
address_df = pd.DataFrame(from_address)
address_df.columns = ["Num_Moves","Postal_Code","Country"]
from_sub = pd.concat([cust_prioritycode, recent_seats, recent_sub_price,
                      total_seats,total_sub_price], axis = 1)
sub_df = pd.DataFrame(from_sub)
sub_df.columns = ["Priority_Code", "Recent_Seat", "Recent_Sub_Price", "Total_Seats","Total_Paid"]

from_orders = pd.concat([num_unique_orders, recent_solicitor, recent_purchase_amt,
                  recent_return_amt,total_purchase_amt, total_return_amt], axis = 1)
orders_df = pd.DataFrame(from_orders)
orders_df.columns = ["Unique_Orders", "Recent_Solicitor", "Recent_Purchase",
                         "Recent_Return", "Total_Purchased","Total_Returned"]

In [194]:
data = (orders_df.join(sub_df, how = 'left')).join(address_df, how = "left")
print(len(data))
data.head()

12791


,Unique_Orders,Recent_Solicitor,Recent_Purchase,Recent_Return,Total_Purchased,Total_Returned,Priority_Code,Recent_Seat,Recent_Sub_Price,Total_Seats,Total_Paid,Num_Moves,Postal_Code,Country
9.0,11,RollOver,1269.2,739.2,NaN,NaN,8.0,0,0.0,10,596.0,5,V6B2S2,32
1009.0,9,RollOver,345.4,188.4,1996.3,852.0,7.0,0,0.0,3,0.0,2,V7V3X8,32
1178.0,1,crowe,39.0,0.0,39.0,0.0,0.0,1,0.0,1,0.0,1,V4N1A2,32
1359.0,5,jason,250.0,125.0,1137.0,311.0,7.0,0,0.0,5,62.0,3,V5T1J9,32
1377.0,1,crowe,102.0,0.0,102.0,0.0,0.0,1,0.0,1,0.0,2,V5N3P9,32


In [198]:
data2 = data.join(t_cust["Churn?"], how = "left")

In [206]:
data3 = data2.join(t_cust["inactive_reason"], how = "left")
data3["inactive_reason"].fillna(0, inplace = True)

In [209]:
final_data = data3
final_data["Churn?"].value_counts()

No     12372
Yes      419
Name: Churn?, dtype: int64

In [215]:
#address_df.to_csv(basepath + "/address_df.csv")
#sub_df.to_csv(basepath + "/sub_df.csv")
#orders_df.to_csv(basepath + "/orders_df.csv")
#final_data.to_csv(basepath + "/final_data.csv")